# **PROCESO ETL** 
- En este notebook se desarrollará la extracción de los datos de la API coingecko, las transformaciones necesarias y la carga en un dataset que permita realizar posteriormente un EDA. 

## Importamos las librerias necesarias

In [20]:
#Librerias para trabajar los datos
import pandas as pd 
import numpy as np 
#Libreria para obtener los datos de la api sin request
#! pip install pycoingecko 
from pycoingecko import CoinGeckoAPI

- El envoltorio para la API de CoinGecko permite llamar a los puntos finales de la API.

In [21]:
#Instanciar la clase CoinGeckoAPI
cg = CoinGeckoAPI() 

## **I. Extracción de los datos**

En la actualidad existen más de 10000 monedas digitales, por lo que es necesario realizar una selección para poder realizar un análisis personalizado. Si bien la selección puede ser arbitraria se seleccionará aquellas monedas en base a las siguientes premisas:
- Una demanda continua asegurando la existencia de un mercado, lo cual se verá con el posicionamiento de actual del trade market en coingecko, escogiendo las 10 primeras de este ranking.
- Una estimación de las cryptos con mejor proyección de crecimiento o se catalogan con mejor futuro, las cuales son detalladas en articulo de la página Investing llamado ["Las criptomonedas con más futuro"](https://es.investing.com/academy/crypto/criptomonedas-con-mas-futuro/), de las cuales se escogeran 2.

1. Selección TOP 10

In [69]:
#El método get_coins_market() devuelve datos relacionados con el mercado
all_coins_market = cg.get_coins_markets(vs_currency ='usd')
#Convertir en dataframe
all_coins_market_df = pd.DataFrame(all_coins_market)
#Seleccionar las 10 cryptos mejor ranqueada.
selected_columns = ['id', 'symbol','name','market_cap_rank']
rank_cryptos = all_coins_market_df[selected_columns]
rank_cryptos.head(10)

,id,symbol,name,market_cap_rank
0,bitcoin,btc,Bitcoin,1
1,ethereum,eth,Ethereum,2
2,tether,usdt,Tether,3
3,binancecoin,bnb,BNB,4
4,ripple,xrp,XRP,5
5,usd-coin,usdc,USD Coin,6
6,staked-ether,steth,Lido Staked Ether,7
7,cardano,ada,Cardano,8
8,dogecoin,doge,Dogecoin,9
9,solana,sol,Solana,10


2. Selección 2 crytos con mayor futuro
- La selección se realizará del articulo ["Las criptomonedas con más futuro"](https://es.investing.com/academy/crypto/criptomonedas-con-mas-futuro/), las crytos son siguientes:
    - Algorand, Cosmos.

In [70]:
#Definir una lista llamada cryptos que contendra todas las monedas a analizar
cryptos = ['bitcoin','ethereum','tether','binancecoin','ripple','usd-coin','staked-ether','cardano','solana','dogecoin','algorand',
'cosmos']
#Imprimir
cryptos_list = rank_cryptos[rank_cryptos['id'].isin(id)]
cryptos_list.head(12)

,id,symbol,name,market_cap_rank
0,bitcoin,btc,Bitcoin,1
1,ethereum,eth,Ethereum,2
2,tether,usdt,Tether,3
3,binancecoin,bnb,BNB,4
4,ripple,xrp,XRP,5
5,usd-coin,usdc,USD Coin,6
6,staked-ether,steth,Lido Staked Ether,7
7,cardano,ada,Cardano,8
8,dogecoin,doge,Dogecoin,9
9,solana,sol,Solana,10


3. Extracción de información 
- Primero, se extraerá los datos generales de las criptomonedas para crear una tabla de Id

In [71]:
columns = ['id', 'symbol','name']
coins = pd.DataFrame(cryptos_list[columns])

- Segundo, se extraerá precios en dolares, volumenes de negociación y market cap de un periodo de 5 años para cada una de las 12 cryptos elegidas.

In [87]:
# Diccionario para almacenar los DataFrames de cada moneda
coin_dataframes = {}

In [88]:
# Iterar a través de los id de monedas y obtener los datos
for crypto in cryptos:
    #Se usa el metodo get_coin_market_chart_range_by_id para obtener data historica entre dos intervalos de tiempo
    coin_history = cg.get_coin_market_chart_range_by_id(id=crypto,
                                                         vs_currency='usd',
                                                         from_timestamp='1534291200', # Agosto 15, 2018
                                                         to_timestamp='1692057600', # Agosto 15, 2023
                                                         localization=False)
    coin_dataframes[crypto] = pd.DataFrame(coin_history)

In [111]:
#Ejemplo de df dentro del diccionario
coin_dataframes['bitcoin']

,prices,market_caps,total_volumes,Data
0,"[1534291200000, 6212.2260400556115]","[1534291200000, 106911788926.75307]","[1534291200000, 7179484727.419351]","[1534291200000, 6212.2260400556115, 1069117889..."
1,"[1534377600000, 6267.76870877378]","[1534377600000, 107788817699.53123]","[1534377600000, 6619548180.387301]","[1534377600000, 6267.76870877378, 107788817699..."
2,"[1534464000000, 6309.385226218996]","[1534464000000, 108578823549.00035]","[1534464000000, 5982589406.965]","[1534464000000, 6309.385226218996, 10857882354..."
3,"[1534550400000, 6566.715163377382]","[1534550400000, 113018605465.47177]","[1534550400000, 6206194158.951437]","[1534550400000, 6566.715163377382, 11301860546..."
4,"[1534636800000, 6382.0605914076095]","[1534636800000, 109889851531.17278]","[1534636800000, 5616965845.869809]","[1534636800000, 6382.0605914076095, 1098898515..."
...,...,...,...,...
1822,"[1691712000000, 29423.81891597763]","[1691712000000, 572414205818.4142]","[1691712000000, 8106253519.025447]","[1691712000000, 29423.81891597763, 57241420581..."
1823,"[1691798400000, 29396.8479714205]","[1691798400000, 571976559045.8302]","[1691798400000, 4651950357.636653]","[1691798400000, 29396.8479714205, 571976559045..."
1824,"[1691884800000, 29412.142274584105]","[1691884800000, 572257798803.9086]","[1691884800000, 3994297895.288944]","[1691884800000, 29412.142274584105, 5722577988..."
1825,"[1691971200000, 29284.96971374381]","[1691971200000, 569687130446.7833]","[1691971200000, 4755220448.634491]","[1691971200000, 29284.96971374381, 56968713044..."


- Una vez creado el diccionario con todos los dataframes, se crea uno para cada coin

In [120]:
#Crear los dataframe para cada crypto
bitcoin_df =  pd.DataFrame(coin_dataframes['bitcoin'])
ethereum_df = pd.DataFrame(coin_dataframes['ethereum'])
tether_df = pd.DataFrame(coin_dataframes['tether'])
binancecoin_df = pd.DataFrame(coin_dataframes['binancecoin'])
ripple_df = pd.DataFrame(coin_dataframes['ripple'])
usdcoin_df = pd.DataFrame(coin_dataframes['usd-coin'])
stakedether_df = pd.DataFrame(coin_dataframes['staked-ether'])
cardano_df = pd.DataFrame(coin_dataframes['cardano'])
solana_df = pd.DataFrame(coin_dataframes['solana'])
dogecoin_df = pd.DataFrame(coin_dataframes['dogecoin'])
algorand_df = pd.DataFrame(coin_dataframes['algorand'])
cosmos_df = pd.DataFrame(coin_dataframes['cosmos'])

In [121]:
#Verificamos un dataframe como ejemplo
bitcoin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827 entries, 0 to 1826
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   prices         1827 non-null   object
 1   market_caps    1827 non-null   object
 2   total_volumes  1827 non-null   object
 3   Data           1827 non-null   object
dtypes: object(4)
memory usage: 57.2+ KB


------------------------

## **II. Transformación de los datos**
- Las transformaciones consisten en los siguientes pasos para cada DF: 
    - Extraer los valores de timestamp, prices, volumes y marketcaps.
    - Añadir una columna Id que contenga el id de la coin. 
    - Transformar a formato date la columna timestamp.

**DATOS GENERALES**

In [182]:
#Cambiamos el nombre de las columnas
coins.columns = ['Id', 'Symbol', 'Coin_Name']

In [183]:
coins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 50
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         12 non-null     object
 1   Symbol     12 non-null     object
 2   Coin_Name  12 non-null     object
dtypes: object(3)
memory usage: 384.0+ bytes


**BITCOIN**

In [122]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
bitcoin_df['Data'] = bitcoin_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in bitcoin_df['Data']]
bitcoin_df = pd.DataFrame(new_data, columns=new_columns)
bitcoin_df.insert(0, 'Id', 'bitcoin')

In [128]:
# Cambiar el formato a fecha en la columna Timestamp
bitcoin_df['Date'] = pd.to_datetime(bitcoin_df['Timestamp'], unit='ms')
bitcoin_df.drop(columns = 'Timestamp', inplace = True)

In [129]:
bitcoin_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,bitcoin,6212.226040,7.179485e+09,1.069118e+11,2018-08-15
1,bitcoin,6267.768709,6.619548e+09,1.077888e+11,2018-08-16
2,bitcoin,6309.385226,5.982589e+09,1.085788e+11,2018-08-17
3,bitcoin,6566.715163,6.206194e+09,1.130186e+11,2018-08-18
4,bitcoin,6382.060591,5.616966e+09,1.098899e+11,2018-08-19


**ETHEREUM**

In [131]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
ethereum_df['Data'] = ethereum_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in ethereum_df['Data']]
ethereum_df = pd.DataFrame(new_data, columns=new_columns)
ethereum_df.insert(0, 'Id', 'ethereum')

In [133]:
# Cambiar el formato a fecha en la columna Timestamp
ethereum_df['Date'] = pd.to_datetime(ethereum_df['Timestamp'], unit='ms')
ethereum_df.drop(columns = 'Timestamp', inplace = True)

In [134]:
ethereum_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,ethereum,277.746017,3.134795e+09,2.814378e+10,2018-08-15
1,ethereum,281.141193,2.818580e+09,2.845771e+10,2018-08-16
2,ethereum,286.320221,2.409323e+09,2.901377e+10,2018-08-17
3,ethereum,314.793253,2.930410e+09,3.190058e+10,2018-08-18
4,ethereum,293.176992,2.806352e+09,2.973165e+10,2018-08-19


**TETHER**

In [136]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
tether_df['Data'] = tether_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in tether_df['Data']]
tether_df = pd.DataFrame(new_data, columns=new_columns)
tether_df.insert(0, 'Id', 'tether')

In [137]:
# Cambiar el formato a fecha en la columna Timestamp
tether_df['Date'] = pd.to_datetime(tether_df['Timestamp'], unit='ms')
tether_df.drop(columns = 'Timestamp', inplace = True)

In [138]:
tether_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,tether,1.003787,9.853164e+07,2.447323e+09,2018-08-15
1,tether,1.000076,1.096754e+08,2.434993e+09,2018-08-16
2,tether,0.986833,1.042327e+08,2.405948e+09,2018-08-17
3,tether,0.998589,6.926751e+07,2.433498e+09,2018-08-18
4,tether,1.000052,9.200243e+07,2.439015e+09,2018-08-19


**BINANCE COIN**

In [140]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
binancecoin_df['Data'] = binancecoin_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in binancecoin_df['Data']]
binancecoin_df = pd.DataFrame(new_data, columns=new_columns)
binancecoin_df.insert(0, 'Id', 'binancecoin')

In [141]:
# Cambiar el formato a fecha en la columna Timestamp
binancecoin_df['Date'] = pd.to_datetime(binancecoin_df['Timestamp'], unit='ms')
binancecoin_df.drop(columns = 'Timestamp', inplace = True)

In [142]:
binancecoin_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,binancecoin,9.449202,3.622425e+07,9.264119e+08,2018-08-15
1,binancecoin,9.418463,4.158557e+07,9.211864e+08,2018-08-16
2,binancecoin,9.730853,2.259454e+07,9.538507e+08,2018-08-17
3,binancecoin,10.751076,3.355559e+07,1.053876e+09,2018-08-18
4,binancecoin,9.860936,2.565619e+07,9.668749e+08,2018-08-19


**RIPPLE**

In [144]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
ripple_df['Data'] = ripple_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in ripple_df['Data']]
ripple_df = pd.DataFrame(new_data, columns=new_columns)
ripple_df.insert(0, 'Id', 'ripple')

In [145]:
# Cambiar el formato a fecha en la columna Timestamp
ripple_df['Date'] = pd.to_datetime(ripple_df['Timestamp'], unit='ms')
ripple_df.drop(columns = 'Timestamp', inplace = True)

In [146]:
ripple_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,ripple,0.273937,2.967558e+08,1.078554e+10,2018-08-15
1,ripple,0.281680,3.408424e+08,1.106011e+10,2018-08-16
2,ripple,0.292357,2.449694e+08,1.150398e+10,2018-08-17
3,ripple,0.365943,5.258470e+08,1.439222e+10,2018-08-18
4,ripple,0.326364,4.447622e+08,1.284835e+10,2018-08-19


**USD-COIN**

In [148]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
usdcoin_df['Data'] = usdcoin_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in usdcoin_df['Data']]
usdcoin_df = pd.DataFrame(new_data, columns=new_columns)
usdcoin_df.insert(0, 'Id', 'usd-coin')

In [149]:
# Cambiar el formato a fecha en la columna Timestamp
usdcoin_df['Date'] = pd.to_datetime(usdcoin_df['Timestamp'], unit='ms')
usdcoin_df.drop(columns = 'Timestamp', inplace = True)

In [150]:
usdcoin_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,usd-coin,1.006242,31264.420430,0.0,2018-10-05
1,usd-coin,1.001530,20254.712255,0.0,2018-10-06
2,usd-coin,1.001177,49324.690669,0.0,2018-10-07
3,usd-coin,1.001906,47076.728142,0.0,2018-10-08
4,usd-coin,1.001983,55542.215509,0.0,2018-10-09


**STAKED-ETHER**

In [152]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
stakedether_df['Data'] = stakedether_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in stakedether_df['Data']]
stakedether_df = pd.DataFrame(new_data, columns=new_columns)
stakedether_df.insert(0, 'Id', 'staked-ether')

In [153]:
# Cambiar el formato a fecha en la columna Timestamp
stakedether_df['Date'] = pd.to_datetime(stakedether_df['Timestamp'], unit='ms')
stakedether_df.drop(columns = 'Timestamp', inplace = True)

In [154]:
stakedether_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,staked-ether,617.164997,137563.026557,0.000000e+00,2020-12-22
1,staked-ether,617.164997,137563.026557,0.000000e+00,2020-12-23
2,staked-ether,596.767711,67102.259216,4.424223e+06,2020-12-24
3,staked-ether,611.504712,54336.794215,5.651613e+06,2020-12-25
4,staked-ether,624.800420,45933.064541,7.284744e+06,2020-12-26


**CARDANO**

In [157]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
cardano_df['Data'] = cardano_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in cardano_df['Data']]
cardano_df = pd.DataFrame(new_data, columns=new_columns)
cardano_df.insert(0, 'Id', 'cardano')

In [158]:
# Cambiar el formato a fecha en la columna Timestamp
cardano_df['Date'] = pd.to_datetime(cardano_df['Timestamp'], unit='ms')
cardano_df.drop(columns = 'Timestamp', inplace = True)

In [159]:
cardano_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,cardano,0.094587,1.412652e+08,2.942845e+09,2018-08-15
1,cardano,0.094590,1.072624e+08,2.935687e+09,2018-08-16
2,cardano,0.095164,5.752800e+07,2.960109e+09,2018-08-17
3,cardano,0.109913,1.186935e+08,3.419660e+09,2018-08-18
4,cardano,0.099657,1.093238e+08,3.100563e+09,2018-08-19


**SOLANA**

In [162]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
solana_df['Data'] = solana_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in solana_df['Data']]
solana_df = pd.DataFrame(new_data, columns=new_columns)
solana_df.insert(0, 'Id', 'solana')

In [163]:
# Cambiar el formato a fecha en la columna Timestamp
solana_df['Date'] = pd.to_datetime(solana_df['Timestamp'], unit='ms')
solana_df.drop(columns = 'Timestamp', inplace = True)

In [164]:
solana_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,solana,0.957606,9.267267e+07,7.827652e+06,2020-04-11
1,solana,0.784711,4.945869e+07,6.173608e+06,2020-04-12
2,solana,0.875994,3.782101e+07,7.269680e+06,2020-04-13
3,solana,0.786712,1.957921e+07,6.302364e+06,2020-04-14
4,solana,0.666673,1.752353e+07,5.306221e+06,2020-04-15


**DOGECOIN**

In [167]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
dogecoin_df['Data'] = dogecoin_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in dogecoin_df['Data']]
dogecoin_df = pd.DataFrame(new_data, columns=new_columns)
dogecoin_df.insert(0, 'Id', 'dogecoin')

In [168]:
# Cambiar el formato a fecha en la columna Timestamp
dogecoin_df['Date'] = pd.to_datetime(dogecoin_df['Timestamp'], unit='ms')
dogecoin_df.drop(columns = 'Timestamp', inplace = True)

In [169]:
dogecoin_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,dogecoin,0.002258,1.018640e+07,2.612948e+08,2018-08-15
1,dogecoin,0.002307,4.062750e+06,2.669251e+08,2018-08-16
2,dogecoin,0.002316,5.509242e+06,2.648706e+08,2018-08-17
3,dogecoin,0.002496,4.174969e+06,2.890354e+08,2018-08-18
4,dogecoin,0.002404,8.131350e+06,2.781876e+08,2018-08-19


**ALGORAND**

In [172]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
algorand_df['Data'] = algorand_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in algorand_df['Data']]
algorand_df = pd.DataFrame(new_data, columns=new_columns)
algorand_df.insert(0, 'Id', 'algorand')

In [173]:
# Cambiar el formato a fecha en la columna Timestamp
algorand_df['Date'] = pd.to_datetime(algorand_df['Timestamp'], unit='ms')
algorand_df.drop(columns = 'Timestamp', inplace = True)

In [174]:
algorand_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,algorand,1.604939,6.653181e+05,0.000000e+00,2019-06-21
1,algorand,3.172364,1.790331e+08,1.010855e+08,2019-06-22
2,algorand,2.159165,2.588226e+08,7.665618e+07,2019-06-23
3,algorand,1.899598,2.100211e+08,6.647512e+07,2019-06-24
4,algorand,1.409310,1.574706e+08,5.522254e+07,2019-06-25


**COSMOS**

In [177]:
# Crear una nueva fila con el timestamp y los valores de cada métrica
cosmos_df['Data'] = cosmos_df.apply(
    lambda row: [row['prices'][0],row['prices'][1], row['total_volumes'][1], row['market_caps'][1]],
    axis=1
)

# Crear un nuevo DataFrame con las columnas Id, Timestamp, Prices, Volumes,Valor_MarketCap
new_columns = ['Timestamp', 'Prices', 'Volumes', 'Market_Cap']
new_data = [list(row) for row in cosmos_df['Data']]
cosmos_df = pd.DataFrame(new_data, columns=new_columns)
cosmos_df.insert(0, 'Id', 'cosmos')

In [178]:
# Cambiar el formato a fecha en la columna Timestamp
cosmos_df['Date'] = pd.to_datetime(cosmos_df['Timestamp'], unit='ms')
cosmos_df.drop(columns = 'Timestamp', inplace = True)

In [179]:
cosmos_df.head()

,Id,Prices,Volumes,Market_Cap,Date
0,cosmos,2.751586,9264.589772,0.0,2019-02-24
1,cosmos,3.002397,528.421839,0.0,2019-02-25
2,cosmos,3.740964,10766.494684,0.0,2019-02-26
3,cosmos,3.992312,13527.310484,0.0,2019-02-27
4,cosmos,3.528405,3560.407725,0.0,2019-02-28


--------------------

## **III. LOAD - CREACIÓN DE DATASETS**
- En este punto los datos estan listos para ser guardados en archivos csv para ser explorados y analizados. 
- Se crea un dataset de indices generales, un dataset para cada coin y un dataset general que unifica todos los datasets independientes.  

In [189]:
#Dataset Indice con datos generales
coins.to_csv('coins_names.csv', index=False)

#Datasets para los datos de cada coin
bitcoin_df.to_csv('bitcoin_df.csv', index=False)
ethereum_df.to_csv('ethereum_df.csv', index=False)
tether_df.to_csv('tether_df.csv', index=False)
binancecoin_df.to_csv('binancecoin_df.csv', index=False)
ripple_df.to_csv('ripple_df.csv', index=False)
usdcoin_df.to_csv('usdcoin_df.csv', index=False)
stakedether_df.to_csv('stakedether_df.csv', index=False)
cardano_df.to_csv('cardano_df.csv', index=False)
solana_df.to_csv('solana_df.csv', index=False)
dogecoin_df.to_csv('dogecoin_df.csv', index=False)
algorand_df.to_csv('algorand_df.csv', index=False)
cosmos_df.to_csv('cosmos_df.csv', index=False)

- Se crea un dataframe consolidado 

In [186]:
#Lista de DF
df_list = [bitcoin_df, ethereum_df, tether_df, binancecoin_df, ripple_df, usdcoin_df, stakedether_df, cardano_df, solana_df,
          dogecoin_df, algorand_df, cosmos_df]

# Concatenar los DataFrames en uno solo
combined_df = pd.concat(df_list)

# Restablecer los índices del DataFrame combinado
combined_df.reset_index(drop=True, inplace=True)


In [187]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19904 entries, 0 to 19903
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Id          19904 non-null  object        
 1   Prices      19904 non-null  float64       
 2   Volumes     19904 non-null  float64       
 3   Market_Cap  19904 non-null  float64       
 4   Date        19904 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 777.6+ KB


In [190]:
#Se guarda el dataframe en un archivo csv
combined_df.to_csv('cryptos_total.csv', index=False)

-------------------------------------